In [1]:
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
import folium
import pandas as pd

Библиотека не обсуждавшаяся в курсе, просто адаптировал.

Получаю координаты по названию места:

In [2]:

geolocator = Nominatim(user_agent="user")
location = geolocator.geocode("Москва", language="ru")
location.point

Point(55.7504461, 37.6174943, 0.0)

In [61]:
url = "https://raw.githubusercontent.com/pinkunicorn322/Final-project-DS/master/postdf.csv"
postdf = pd.read_csv(url).drop(columns=["Unnamed: 0"])
postdf

city_from          city_to  weight
0     Ростов-на-Дону              Бор   0.300
1             Москва        Волгоград   5.000
2               Сочи           Москва   0.060
3           Добрянка            Пермь   0.100
4          Зерноград  Санкт-Петербург   0.500
...              ...              ...     ...
2616          Москва           Москва   0.075
2617          Москва      Калининград   0.043
2618          Москва         Улан-Удэ   0.029
2619          Москва  Санкт-Петербург   0.033
2620          Москва  Санкт-Петербург   0.057

[2621 rows x 3 columns]

Извиняюсь за длинный вывод. Следующий блок кода агреггирует все посылки в словарь, который представляет из себя ненаправленный граф.

In [62]:
D = {}
for i in postdf.index:
    fr, to, w = postdf.iloc[i, :]
    wedge = tuple(sorted((fr, to)))
    if wedge[0] not in D:
        D[wedge[0]] = {}
        D[wedge[0]][wedge[1]] = w
    else:
        if wedge[1] not in D[wedge[0]]:
            D[wedge[0]][wedge[1]] = w
        else:
            D[wedge[0]][wedge[1]] += w
D

{'Бор': {'Ростов-на-Дону': 0.3},
 'Волгоград': {'Москва': 8.111,
  'Ростов-на-Дону': 3.0,
  'Саратов': 5.5,
  'Набережные Челны': 1.0,
  'Нижний Новгород': 23.46,
  'Ижевск': 0.7,
  'Тула': 0.5,
  'Дзержинский': 2.0,
  'Ульяновск': 0.5,
  'Чита': 2.0,
  'Волгоград': 5.0,
  'Урюпинск': 3.0,
  'Михайловка': 1.0,
  'Камышин': 2.0,
  'Котово': 1.0,
  'Краснодар': 2.0,
  'Санкт-Петербург': 0.4,
  'Химки Новые': 1.0},
 'Москва': {'Сочи': 9.724,
  'Рязань': 13.02,
  'Новокузнецк': 2.533,
  'Ярославль': 7.243,
  'Санкт-Петербург': 89.62500000000009,
  'Ступино': 1.4,
  'Москва': 68.38900000000002,
  'Ставрополь': 4.94,
  'Рыбинск': 6.22,
  'Пятигорск': 1.2000000000000002,
  'Павловский Посад': 5.45,
  'Саратов': 2.7479999999999998,
  'Селятино': 1.2,
  'Некрасовка район': 1.5,
  'Тамбов': 2.379,
  'Петрозаводск': 0.2,
  'Чебоксары': 0.7,
  'Северодвинск': 1.9,
  'Сиверский, Гатчинский р-н': 0.3,
  'Серпухов': 2.0,
  'Усолье-Сибирское': 0.441,
  'Подольск': 0.29400000000000004,
  'Реутов': 0.93

Следующий блок кода выполняется долго(!). Я не стал его перезапускать. Его задача в том, чтобы удалить рёбра с весом <= 2, а также удалить вершины, координаты которых не может найти geopy. Таких координат не много, в основном это небольшие поселения. Как видно, выводил эти места. sys.getsizeof(D) показывает размер словаря в байтах. Я хотел уменьшить D, чтобы карта быстрее рисовалась

In [70]:
for i in list(D.keys()):
    location = geolocator.geocode(i, language="ru")
    
    if location is None:
        del D[i]
        print(i)
        continue
    
    for j in list(D[i].keys()):
        location = geolocator.geocode(j, language="ru")
        if location is None:
            del D[i][j]
            print(j)
            continue
        if D[i][j] <= 2:
            del D[i][j]
sys.getsizeof(D)       

Сиверский, Гатчинский р-н
Новоселье, Ломоносовский р-он
Смольки, Городецкий р-н
Семилуки, Семилукский р-н
Ртищево, Ртищевский р-он
Новосиньково, Дмитровский р-н
Румянцево, Московский пос.
Солнцево, р-н
Шебекино, Шебекинский р-н
Майский, Белгородский р-н
Цемдолина, гор. окр Новороссийск
Семилуки, Семилукский р-н
Ямное, Рамонский р-н
Лиски, Лискинский р-н
Острогожск, Острогожский р-н
Суханово, Ленинский гор.округ
Солнцево, р-н
Тимашевск, Тимашевский р-н
Киров, Кировский р-н
Московский, Тюменский р-н
Миллерово, Миллеровский р-н
Октябрьский, Башкортостан респ.
Коченёво, Коченёвский р-н
Алексеевка, Алексеевский р-н
Васильево, Зеленодольский р-он
Весёлое, Сочи гор.округ
Юбилейный мкр. (Королёв)
Менделеево, Солнечногорский р-он
Румянцево, Московский пос.
Ильский, Северский р-н
Горбунки, Ломоносовский р-н
Гонконг (Чхеклапкок)


9312

И наконец карта. Чем толще ребро, тем больше по массе был объём сдэк перевозок. Как мне кажется, это красиво, и может быть полезно, например, при анализе сферы доставки.

In [75]:
m = folium.Map([55.7504461, 37.6174943], zoom_start=5)
for i in D.keys():
    location = geolocator.geocode(i, language="ru")
#     lat1, lon1 = location.latitude, location.longitude
    if location is None:
        print(i)
        continue
    point1 = tuple([location.latitude, location.longitude])
    
    for j in D[i].keys():
        location = geolocator.geocode(j, language="ru")
        if location is None:
            print(j)
            continue
#         lat2, lon2 = location.latitude, location.longitude
        point2 = tuple([location.latitude, location.longitude])
        folium.PolyLine([point1, point2], color="blue", weight=D[i][j]/5, opacity=0.5).add_to(m)
#         folium.Circle([location.latitude, location.longitude], radius=50).add_to(m)
m

## Итоги: помощь в проверке